In [1]:
# !pip install pyspark py4j

In [2]:
import os
# The latest version of spark 3.2  from http://www.apache.org/dist/spark/ 
spark_version = 'spark-3.2.3'
# spark_version = 'spark-3.'
os.environ['SPARK_VERSION']=spark_version
# PYSPARK_PYTHON = python3.8


# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (78.0 kB/s)
Reading package li

In [3]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-12 20:29:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.74MB/s    in 0.2s    

2022-12-12 20:29:53 (5.74 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdata-amazon").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# from posixpath import sep
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

outdoors = spark.read.csv(SparkFiles.get ("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), sep="\t", header=True)


In [6]:
outdoors.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [7]:
# Get the number of rows in the DataFrame.
outdoors_df = outdoors.dropna()

outdoors_df.count()

2302174

In [8]:
# outdoors_df.dtypes
outdoors_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# Transform the Data

## Create the "review_id_table".

In [9]:
from pyspark.sql.types import IntegerType, DateType
outdoors_df = outdoors_df.withColumn("customer_id", outdoors_df["customer_id"].cast(IntegerType()))
outdoors_df = outdoors_df.withColumn("product_parent", outdoors_df["product_parent"].cast(IntegerType()))
outdoors_df = outdoors_df.withColumn("review_date", outdoors_df["review_date"].cast(DateType()))
outdoors_df = outdoors_df.withColumn("helpful_votes", outdoors_df["helpful_votes"].cast(IntegerType()))
outdoors_df = outdoors_df.withColumn("total_votes", outdoors_df["total_votes"].cast(IntegerType()))
outdoors_df = outdoors_df.withColumn("star_rating", outdoors_df["star_rating"].cast(IntegerType()))


In [10]:
outdoors_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [21]:
review_id = outdoors_df.select(["review_id","customer_id","product_id","product_parent", "review_date"])
review_id.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R35T75OLUGHL5C|   18446823|B000NV6H94|     110804376| 2015-08-31|
|R2BV735O46BN33|   13724367|B000IN0W3Y|     624096774| 2015-08-31|
|R2NBEUGPQQGXP1|   51001958|B008RBJXFM|     278970944| 2015-08-31|
|R17LLAOJ8ITK0S|   32866903|B00FK8WUQY|     312877650| 2015-08-31|
|R39PEQBT5ISEF4|   30907790|B00EZA3VW0|     305567912| 2015-08-31|
|R3GNM3SU9VHJFT|   20232229|B006JA8WEG|     842306035| 2015-08-31|
| R2Y81OP0EK467|   17698862|B002PWFSEO|     451480122| 2015-08-31|
|R2LFGSI6HAYH5F|   38486114|B002DZGKHW|     124386306| 2015-08-31|
|R297G6ED1IQO7W|   26319572|B00ABA08F6|     991442421| 2015-08-31|
| RE27RFC6101N6|   27152337|B003Z8WIHC|     886483892| 2015-08-31|
|R3BPDME6E94W8Z|   12516845|B007CP6UK0|     150224054| 2015-08-31|
|R2P08O1RILUOX3|    3225242|B003V3U9JK|     343847969| 2015-08

## Create the "products" Table

In [22]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products = outdoors_df.select(["product_id","product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000NV6H94|Stearns Youth Boa...|
|B000IN0W3Y|Primal Wear Men's...|
|B008RBJXFM|Osprey Hydraulics...|
|B00FK8WUQY|CamelBak eddy .75...|
|B00EZA3VW0|Children Black Re...|
|B006JA8WEG|Ibera Bicycle Tri...|
|B002PWFSEO|Therm-a-Rest Comp...|
|B002DZGKHW|Sawyer Products P...|
|B00ABA08F6|Zippo Hand Warmer...|
|B003Z8WIHC|Camp Chef Dutch O...|
|B007CP6UK0|3CERA Portable Wi...|
|B003V3U9JK|Texsport King Kot...|
|B00Y846HN8|Wallygadgets 2 Wh...|
|B00IYQ84VY|RainStoppers 34-I...|
|B002MYCKLY|Alpha Deluxe Port...|
|B00EZV69JG|Speedfil Z4 BTA B...|
|B00AWOT3T8|O'Brien Kids Plat...|
|B0000AXTID|Kwik-Tek F-5R Pla...|
|B00IFHFJXI|Ivation Portable ...|
|B00KFILTWU|GreenInsync Repla...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [23]:
outdoors_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [25]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers = outdoors_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count", "customer_count")
customers.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   43679767|                 1|
|   32024654|                 1|
|   52913169|                 1|
|   24297214|                 1|
|   26096454|                 1|
|   38247118|                 1|
|   44009906|                 1|
|    1753876|                 1|
|   26195644|                 1|
|   23042837|                 1|
|   11487525|                 2|
|   28258386|                 1|
|   42560427|                 2|
|   24540309|                 1|
|    1967239|                 1|
|   14217455|                 1|
|   20289484|                26|
|   47103434|                 1|
|   28638887|                 1|
|     740678|                 1|
+-----------+------------------+
only showing top 20 rows



## Create the "vine_table".

In [26]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine = outdoors.select(['review_id','star_rating', 'helpful_votes', 'vine'])
vine.show()

+--------------+-----------+-------------+----+
|     review_id|star_rating|helpful_votes|vine|
+--------------+-----------+-------------+----+
|R35T75OLUGHL5C|          4|            0|   N|
|R2BV735O46BN33|          5|            0|   N|
|R2NBEUGPQQGXP1|          4|            0|   N|
|R17LLAOJ8ITK0S|          3|            1|   N|
|R39PEQBT5ISEF4|          1|            0|   N|
|R3GNM3SU9VHJFT|          4|            1|   N|
| R2Y81OP0EK467|          5|            0|   N|
|R2LFGSI6HAYH5F|          5|            1|   N|
|R297G6ED1IQO7W|          5|            1|   N|
| RE27RFC6101N6|          5|            0|   N|
|R3BPDME6E94W8Z|          5|            0|   N|
|R2P08O1RILUOX3|          3|            0|   N|
|R37CVAB03PTDVI|          5|            0|   N|
| RAWNWOGXPCPMD|          5|            0|   N|
| R5DYGP6ASX77M|          5|            0|   N|
|R1O0SAOOGF2KG7|          4|            0|   N|
|R35NJUT0U3MU3V|          5|            0|   N|
|R242C08MF9D1AH|          5|            

# Load

In [34]:
# import config
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/bigdata-amazon"
config = {"user":"", 
          "password": "", 
          "driver":"org.postgresql.Driver"}
     

In [ ]:
# Write review_id_df to table in RDS
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [33]:
# Write products_df to table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [18]:
# Write customers_df to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)


In [19]:
# Write vine_df to table in RDS
vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
